<a href="https://colab.research.google.com/github/nurkbts/yenidepom/blob/master/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import vstack,argmax
from pandas import read_csv
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import Dataset,DataLoader,random_split
from torch.nn import *
import pandas as pd 
import torch.nn as nn
import torch.optim as optim
from loss import OZELoss
from transformer import Transformer
import seaborn as sns
from tqdm import tqdm
import datetime
from utils_ import compute_loss
#from plot_functions import map_plot_function, plot_values_distribution, plot_error_distribution, plot_errors_threshold, plot_visual_sampl
class CSVDataset(Dataset):
    def __init__(self, path):
        KINEMATICS_USECOLS = [c-1 for c in [39, 40, 41, 51, 52, 53, 57, 58, 59, 60, 70, 71, 72, 76,77]]
        df = read_csv(path,sep=',',usecols=KINEMATICS_USECOLS)
        self.X, self.y = df.values[:, :-1], df.values[:, -1]
        self.X, self.y = self.X.astype('float32'), LabelEncoder().fit_transform(self.y)
        #veri normalizasyonu
        mu =  self.X.mean(axis=0, keepdims=True)
        sigma =  self.X.std(axis=0, keepdims=True)
        self.X = ( self.X - mu) / sigma 
        self.X= torch.Tensor(self.X)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
    #  if torch.is_tensor(idx):
     #  idx = idx.tolist()
      return [self.X[idx], self.y[idx]]
def prepare_data(path):
    dataset = CSVDataset(path)
    train_dl  = DataLoader(dataset, batch_size=BATCH_SIZE,shuffle=True, num_workers=NUM_WORKERS,pin_memory=False)
    return train_dl
BATCH_SIZE =8
NUM_WORKERS= 0
LR =0.001
EPOCHS=20
d_model =64
q=8 
v=8
h = 8 
N = 4
attention_size = 12
dropout = 0.2
pe = None
chunk_mode = None 
d_input = 14
d_output= 10
path = 'cdefg.txt'
train_dl = prepare_data(path)
dataset_test =CSVDataset('b.txt')
test_dl = DataLoader(dataset_test, batch_size=BATCH_SIZE,shuffle=False, num_workers=NUM_WORKERS)
print(len(train_dl.dataset),  len(test_dl.dataset))
sns.set()
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")

net = Transformer(d_input, d_model, d_output, q, v, h, N, attention_size=attention_size, dropout=dropout, chunk_mode=chunk_mode, pe=pe)
#optimizer = optim.Adam(net.parameters(), lr=LR)
optimizer = optim.SGD(net.parameters(), lr=LR,weight_decay=0.5, momentum=0.9)
#optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9)
loss_function = nn.CrossEntropyLoss() 
model_save_path = f'models/model_{datetime.datetime.now().strftime("%Y_%m_%d__%H%M%S")}.pth'
for idx_epoch in range(EPOCHS):
    running_loss = 0
    with tqdm(total=len(train_dl.dataset), desc=f"[Epoch {idx_epoch+1:3d}/{EPOCHS}]") as pbar:
      for idx_batch, (x, y) in enumerate(train_dl):
        optimizer.zero_grad()
        netout = net(x)
        loss = loss_function(netout, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        pbar.set_postfix({'loss': running_loss/(idx_batch+1)})
        pbar.update(x.shape[0])

# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        yhat = model(inputs)
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        yhat = argmax(yhat, axis=1)
        actual = actual.reshape((len(actual), 1))
        yhat = yhat.reshape((len(yhat), 1))
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    acc = accuracy_score(actuals, predictions)
    return acc

acc = evaluate_model(test_dl, net)
print('Accuracy: %.3f' % acc)


[Epoch   1/20]:   0%|          | 0/74725 [00:00<?, ?it/s]

74725 18354
Using device cpu


[Epoch   1/20]:  70%|███████   | 52656/74725 [08:18<02:56, 125.16it/s, loss=2.3]

In [ ]:
from numpy import vstack,argmax
from pandas import read_csv
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import Dataset,DataLoader,random_split
from torch.nn import *
import pandas as pd 
import torch.nn as nn
import torch.optim as optim
from loss import OZELoss
from transformer import Transformer
import seaborn as sns
from tqdm import tqdm
import datetime
from utils_ import compute_loss
#from plot_functions import map_plot_function, plot_values_distribution, plot_error_distribution, plot_errors_threshold, plot_visual_sampl
class CSVDataset(Dataset):
    def __init__(self, path):
        KINEMATICS_USECOLS = [c-1 for c in [39, 40, 41, 51, 52, 53, 57, 58, 59, 60, 70, 71, 72, 76]]
        train = read_csv('x.txt',sep=',',usecols=KINEMATICS_USECOLS)
        test = read_csv('_y.txt')
        self.X, self.y = df.values[:, :-1], df.values[:, -1]
        self.X, self.y = train.values, test.values
        print(self.X.shape)
        self.X = self.X.astype('float32')
        self.y =  LabelEncoder().fit_transform(self.y)
        self.X= torch.Tensor(self.X)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
    #  if torch.is_tensor(idx):
     #  idx = idx.tolist()
      return [self.X[idx], self.y[idx]]
    def get_splits(self, n_test=0.33):
        test_size = round(n_test * len(self.X))
        val_size =1000
        train_size = len(self.X) - test_size-1000
        return random_split(self, [train_size, val_size, test_size])
def prepare_data(path):
    dataset = CSVDataset(path)
    train, val, test = dataset.get_splits()
    train_dl ,val_dl,test_dl = DataLoader(train, batch_size=BATCH_SIZE,shuffle=True, num_workers=NUM_WORKERS,pin_memory=False),DataLoader(val, batch_size=BATCH_SIZE,shuffle=False,num_workers=NUM_WORKERS),DataLoader(test, batch_size=BATCH_SIZE,shuffle=False, num_workers=NUM_WORKERS)
    return train_dl, val_dl, test_dl
BATCH_SIZE ,NUM_WORKERS, LR,EPOCHS,d_model ,q ,v, h, N,attention_size,dropout,pe,chunk_mode,d_input,d_output= 8,0,2e-4,20,64,8 ,8,8,4,16 ,0.2,None,None,14,10
path = 'merge_from_ofoct (1).txt'
train_dl, val_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(val_dl.dataset),  len(test_dl.dataset))
sns.set()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")
net = Transformer(d_input, d_model, d_output, q, v, h, N, attention_size=attention_size, dropout=dropout, chunk_mode=chunk_mode, pe=pe).to(device)
optimizer = optim.Adam(net.parameters(), lr=LR)
loss_function = nn.CrossEntropyLoss() 
#loss_function = OZELoss(alpha=0.3)
model_save_path = f'models/model_{datetime.datetime.now().strftime("%Y_%m_%d__%H%M%S")}.pth'
for idx_epoch in range(EPOCHS):
    running_loss = 0
    with tqdm(total=len(train_dl.dataset), desc=f"[Epoch {idx_epoch+1:3d}/{EPOCHS}]") as pbar:
      for idx_batch, (x, y) in enumerate(train_dl):
        optimizer.zero_grad()
        #print(idx_batch)
        netout = net(x.to(device))
        #print(netout.size())
        #print("SSSSSSSS")
        #print(y.size())
        loss = loss_function(netout, y)
        #print("eee")
        loss.backward()
        #print("kkkk")
        optimizer.step()
        running_loss += loss.item()
        pbar.set_postfix({'loss': running_loss/(idx_batch+1)})
        pbar.update(x.shape[0])

# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        # convert to class labels
        yhat = argmax(yhat, axis=1)
        # reshape for stacking
        actual = actual.reshape((len(actual), 1))
        yhat = yhat.reshape((len(yhat), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc

acc = evaluate_model(test_dl, net)
print('Accuracy: %.3f' % acc)


NameError: ignored

In [ ]:
#VAL OLMADAN
from numpy import vstack,argmax
from pandas import read_csv
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import Dataset,DataLoader,random_split
from torch.nn import *
import pandas as pd 
import torch.nn as nn
import torch.optim as optim
from loss import OZELoss
from transformer import Transformer
import seaborn as sns
from tqdm import tqdm
import datetime
from utils_ import compute_loss
#from plot_functions import map_plot_function, plot_values_distribution, plot_error_distribution, plot_errors_threshold, plot_visual_sampl
class CSVDataset(Dataset):
    def __init__(self, path):
        KINEMATICS_USECOLS = [c-1 for c in [39, 40, 41, 51, 52, 53, 57, 58, 59, 60, 70, 71, 72, 76]]
        train = read_csv('x.txt',sep=',',usecols=KINEMATICS_USECOLS)
        test = read_csv('_y.txt')
        #self.X, self.y = df.values[:, :-1], df.values[:, -1]
        self.X, self.y = train.values, test.values
        print(self.X.shape)
        self.X = self.X.astype('float32')
        self.y =  LabelEncoder().fit_transform(self.y)
        self.X= torch.Tensor(self.X)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
    #  if torch.is_tensor(idx):
     #  idx = idx.tolist()
      return [self.X[idx], self.y[idx]]
    def get_splits(self, n_test=0.33):
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        return random_split(self, [train_size, test_size])
def prepare_data(path):
    dataset = CSVDataset(path)
    train, val, test = dataset.get_splits()
    train_dl ,val_dl,test_dl = DataLoader(train, batch_size=BATCH_SIZE,shuffle=True, num_workers=NUM_WORKERS,pin_memory=False),DataLoader(val, batch_size=BATCH_SIZE,shuffle=False,num_workers=NUM_WORKERS),DataLoader(test, batch_size=BATCH_SIZE,shuffle=False, num_workers=NUM_WORKERS)
    return train_dl, val_dl, test_dl
BATCH_SIZE ,NUM_WORKERS, LR,EPOCHS,d_model ,q ,v, h, N,attention_size,dropout,pe,chunk_mode,d_input,d_output= 8,0,2e-4,20,64,8 ,8,8,4,16 ,0.2,None,None,14,10
path = 'train.txt'
train_dl, val_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(val_dl.dataset),  len(test_dl.dataset))
sns.set()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")
net = Transformer(d_input, d_model, d_output, q, v, h, N, attention_size=attention_size, dropout=dropout, chunk_mode=chunk_mode, pe=pe).to(device)
optimizer = optim.Adam(net.parameters(), lr=LR)
loss_function = nn.CrossEntropyLoss() 
#loss_function = OZELoss(alpha=0.3)
model_save_path = f'models/model_{datetime.datetime.now().strftime("%Y_%m_%d__%H%M%S")}.pth'
for idx_epoch in range(EPOCHS):
    running_loss = 0
    with tqdm(total=len(train_dl.dataset), desc=f"[Epoch {idx_epoch+1:3d}/{EPOCHS}]") as pbar:
      for idx_batch, (x, y) in enumerate(train_dl):
        optimizer.zero_grad()
        #print(idx_batch)
        netout = net(x.to(device))
        #print(netout.size())
        #print("SSSSSSSS")
        #print(y.size())
        loss = loss_function(netout, y)
        #print("eee")
        loss.backward()
        #print("kkkk")
        optimizer.step()
        running_loss += loss.item()
        pbar.set_postfix({'loss': running_loss/(idx_batch+1)})
        pbar.update(x.shape[0])

# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        # convert to class labels
        yhat = argmax(yhat, axis=1)
        # reshape for stacking
        actual = actual.reshape((len(actual), 1))
        yhat = yhat.reshape((len(yhat), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc

acc = evaluate_model(test_dl, net)
print('Accuracy: %.3f' % acc)


ValueError: ignored

In [ ]:
import pandas as pd
import numpy as np
def x():
  
  from sklearn.preprocessing import MinMaxScaler
  from scipy.stats import zscore
  KINEMATICS_USECOLS = [c-1 for c in [39, 40, 41, 51, 52, 53, 57, 58, 59, 60, 70, 71, 72, 76]]
  trainX = []
  filenamesTrainX = ['B001_X.txt','B002_X.txt','B003_X.txt','B004_X.txt','B005_X.txt',
                    'C001_X.txt','C002_X.txt','C003_X.txt','C004_X.txt','C005_X.txt',
                    'D001_X.txt','D002_X.txt','D003_X.txt','D004_X.txt','D005_X.txt',
                    'E001_X.txt','E002_X.txt','E003_X.txt','E004_X.txt','E005_X.txt',
                    'F001_X.txt','F002_X.txt','F003_X.txt','F004_X.txt','F005_X.txt',       
                    'G001_X.txt','G002_X.txt','G003_X.txt','G004_X.txt','G005_X.txt',
                    'H001_X.txt','H003_X.txt','H004_X.txt','H005_X.txt']

  for fname in filenamesTrainX:
      trainXdata = pd.read_csv(fname,sep=',',usecols=KINEMATICS_USECOLS)
      trainXdata=trainXdata.values
      trainXdata=trainXdata.astype(np.float32)
      trainX.extend(trainXdata)

  trainX=np.array(trainX)
  print(trainX.shape)
  return trainX


In [ ]:
from sklearn.preprocessing import MinMaxScaler
def y():
    trainY = []
    testY2 = []
    filenamesTrainY = ['B001_y.txt','B002_y.txt','B003_y.txt','B004_y.txt','B005_y.txt',
                    'C001_y.txt','C002_y.txt','C003_y.txt','C004_y.txt','C005_y.txt',
                    'D001_y.txt','D002_y.txt','D003_y.txt','D004_y.txt','D005_y.txt',
                    'E001_y.txt','E002_y.txt','E003_y.txt','E004_y.txt','E005_y.txt',
                    'F001_y.txt','F002_y.txt','F003_y.txt','F004_y.txt','F005_y.txt',
                    'G001_y.txt','G002_y.txt','G003_y.txt','G004_y.txt','G005_y.txt',
                    'H001_y.txt','H003_y.txt','H004_y.txt','H005_y.txt'
                    ]

    for fname in filenamesTrainY:
      KINEMATICS_USECOLS = [c-1 for c in [39, 40, 41, 51, 52, 53, 57, 58, 59, 60, 70, 71, 72, 76,77]]
      testY2 = []
      trainYdata = pd.read_csv(fname)
      trainYdata=trainYdata.values
      trainY.extend(trainYdata)
      trainY=np.array(trainY)
      print(trainY.shape)
      return trainY
